In [1]:
#Initial environement of computations

import matplotlib.pyplot as plt
import numpy as np
from kanon.calendars import Calendar, Date
from kanon.calendars.calendars import float_to_hm, hm_to_float
from kanon.tables import HTable
from kanon.units import Sexagesimal, Historical
from kanon.units.precision import set_precision, TruncatureMode, PrecisionMode
import astropy.units as u


from kanon.tables.htable_reader import (
    DISHAS_REQUEST_URL,
    read_historical,
    read_sexag_array,
    read_intsexag_array,
)
from kanon.tables import HTable
import requests


number_reader = {
    "sexagesimal": read_sexag_array,
    "floating sexagesimal": read_sexag_array,
    "integer and sexagesimal": read_intsexag_array,
    "historical": read_historical,
}

In [2]:
#Initial table set

# Historical target table
Hist_table=HTable.read(1448, with_units=False)
Hist_table


Years,Entries
IntegerAndSexagesimal,Historical
365 ;,"19 ; 19,41,54"
730 ;,"01s 08 ; 39,29,48"
1095 ;,"01s 27 ; 59,05,42"
1461 ;,"02s 17 ; 21,58,14"
1826 ;,"03s 08 ; 41,40,09"
2191 ;,"03s 26 ; 01,22,03"
2556 ;,"04s 15 ; 21,03,57"
2922 ;,"05s 04 ; 43,56,24"
3287 ;,"05s 24 ; 03,38,23"


In [3]:
# Formating and initialising the Test tables

Test_recomp=Hist_table.copy()
Delta_recomp=Hist_table.copy()
for i in range(34):
    Test_recomp[i][1]=Historical(Sexagesimal('0;0,0'),2)
    Delta_recomp[i][1]=Historical(Sexagesimal('0;0,0'),2)

In [4]:
# mean motion parameter from the Parisian Alfonsine tables
daily_mean=Historical(Sexagesimal("0;3,10,38,7,14,49,10"),7)
REVO=Historical(Sexagesimal("6,0;0,0"),7)

for i in range(34):
    with set_precision(tmode=TruncatureMode.TRUNC, pmode=7):
        days=Hist_table[i][0]
        Test_recomp[i][1]=(daily_mean*days)%REVO
        with set_precision(tmode=TruncatureMode.ROUND, pmode=3):
            Test_recomp[i][1]=1*Test_recomp[i][1]
            Delta_recomp[i][1]=60*60*60*(Hist_table[i][1]-Test_recomp[i][1])

In [5]:
# residuals visualisation and statistical summaries 1
B=Delta_recomp.to_pandas()
(B.mean().mean(), B.std().std())

(70097.4705882353, nan)

In [6]:
B.astype(float).style.background_gradient(axis=None)

,Entries
Years,
365 ;,0.000000
730 ;,360.000000
1095 ;,0.000000
1461 ;,0.000000
1826 ;,432000.000000
2191 ;,0.000000
2556 ;,0.000000
2922 ;,-5.000000
3287 ;,0.000000


In [8]:
#csv export of the recomputed table
B=Test_recomp.to_pandas()
compression_opts = dict(method='zip',archive_name='Lunar node years.csv')
B.to_csv('Lunar node years.zip', index=False, compression=compression_opts)